In [1]:
import os
import glob
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from utils_function.sort_files import alphanumeric_sort

# ---------------------------
# Chemins des données
# ---------------------------
PATH = "./data/"
train_path = os.path.join(PATH, "train-images/")

# ---------------------------
# 1. Dataset Definition
# ---------------------------
class CTScanDataset(Dataset):
    def __init__(self, image_dir, mask_csv=None, transform=None):
        self.image_dir = image_dir
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, "*.png")),key = alphanumeric_sort)
        self.transform = transform
        
        if mask_csv is not None:
            # CSV transposé : index=pixelIdx, cols=image_index
            masks = pd.read_csv(mask_csv, index_col=0, header=0).T.values
            self.masks = masks.reshape(-1, 256, 256).astype(np.int64)
        else:
            self.masks = None

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Charger image
        img = Image.open(self.image_paths[idx]).convert("L")
        img = np.array(img, dtype=np.float32) / 255.0
        img_tensor = torch.from_numpy(img).unsqueeze(0)  # (1,H,W)

        mask_tensor = None
        if self.masks is not None:
            mask = self.masks[idx]
            mask_tensor = torch.from_numpy(mask)  # (H,W)

        if self.transform:
            img_tensor = self.transform(img_tensor)

        return img_tensor, mask_tensor

# ---------------------------
# 2. UNet Definition
# ---------------------------
class DoubleConv(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.net(x)

class UNet(nn.Module):
    def __init__(self, in_channels=1, base_filters=32, num_classes=55):
        super().__init__()
        f = base_filters
        # Encoder
        self.down1 = DoubleConv(in_channels, f)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = DoubleConv(f, f*2)
        self.pool2 = nn.MaxPool2d(2)
        self.down3 = DoubleConv(f*2, f*4)
        self.pool3 = nn.MaxPool2d(2)
        self.bottleneck = DoubleConv(f*4, f*8)

        # Decoder
        self.up3 = nn.ConvTranspose2d(f*8, f*4, 2, stride=2)
        self.conv_up3 = DoubleConv(f*8, f*4)
        self.up2 = nn.ConvTranspose2d(f*4, f*2, 2, stride=2)
        self.conv_up2 = DoubleConv(f*4, f*2)
        self.up1 = nn.ConvTranspose2d(f*2, f, 2, stride=2)
        self.conv_up1 = DoubleConv(f*2, f)

        # Segmentation head
        self.seg_head = nn.Conv2d(f, num_classes, kernel_size=1)

    def forward(self, x):
        # Encoder
        d1 = self.down1(x)
        p1 = self.pool1(d1)
        d2 = self.down2(p1)
        p2 = self.pool2(d2)
        d3 = self.down3(p2)
        p3 = self.pool3(d3)
        bn = self.bottleneck(p3)

        # Decoder
        u3 = self.up3(bn)
        c3 = self.conv_up3(torch.cat([u3, d3], dim=1))
        u2 = self.up2(c3)
        c2 = self.conv_up2(torch.cat([u2, d2], dim=1))
        u1 = self.up1(c2)
        c1 = self.conv_up1(torch.cat([u1, d1], dim=1))

        seg_logits = self.seg_head(c1)  # (B, num_classes, H, W)
        return seg_logits

# ---------------------------
# 3. Training Loop
# ---------------------------

def train_model(model, dataloader, epochs=20, lr=1e-4, device='cuda'):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        for imgs, masks in tqdm(dataloader):
            imgs = imgs.to(device)
            masks = masks.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)        # (B,55,H,W)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * imgs.size(0)

        avg_loss = total_loss / len(dataloader.dataset)
        print(f"Epoch {epoch}/{epochs} — Loss: {avg_loss:.4f}")




In [ ]:

dataset = CTScanDataset(image_dir=train_path, mask_csv=PATH+"y_train.csv")
loader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)

    # Model
model = UNet(in_channels=1, base_filters=32, num_classes=55)
train_model(model, loader, epochs=20, lr=1e-1, device='cuda')

  0%|                                                                                          | 0/250 [00:00<?, ?it/s]